In [1]:
import os
import cv2
import h5py
import numpy as np
import pandas as pd
import tensorflow as tf

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [5]:
def load_images_from_hdf5(hdf5_path, image_ids):
    with h5py.File(hdf5_path, 'r') as hdf5_file:
        images = []
        for img_id in image_ids:
            image = np.array(hdf5_file[img_id])
            images.append(image)
    return np.array(images)

In [6]:
metadata = pd.read_csv('E:/W/ML NUST/(T5) CNN, RNN, LSTM/isic-2024-data/train-metadata.csv')
subset_metadata = metadata.sample(n=1000, random_state=42)

subset_image_ids = subset_metadata['isic_id'].values
images = load_images_from_hdf5('E:/W/ML NUST/(T5) CNN, RNN, LSTM/isic-2024-data/train-image.hdf5', subset_image_ids)
labels = subset_metadata['target'].values

C:\Users\jahme\AppData\Local\Temp\ipykernel_3512\4053749783.py:1: DtypeWarning: Columns (51,52) have mixed types. Specify dtype option on import or set low_memory=False.
  metadata = pd.read_csv('E:/W/ML NUST/(T5) CNN, RNN, LSTM/isic-2024-data/train-metadata.csv')


In [7]:
datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Example of fitting the data generator
datagen.fit(images)

# Augment and preprocess the images
augmented_images = datagen.flow(images, batch_size=32)

ValueError: could not convert string to float: b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xdb\x00C\x00\x08\x06\x06\x07\x06\x05\x08\x07\x07\x07\t\t\x08\n\x0c\x14\r\x0c\x0b\x0b\x0c\x19\x12\x13\x0f\x14\x1d\x1a\x1f\x1e\x1d\x1a\x1c\x1c $.\' ",#\x1c\x1c(7),01444\x1f\'9=82<.342\xff\xdb\x00C\x01\t\t\t\x0c\x0b\x0c\x18\r\r\x182!\x1c!22222222222222222222222222222222222222222222222222\xff\xc0\x00\x11\x08\x00{\x00{\x03\x01"\x00\x02\x11\x01\x03\x11\x01\xff\xc4\x00\x1f\x00\x00\x01\x05\x01\x01\x01\x01\x01\x01\x00\x00\x00\x00\x00\x00\x00\x00\x01\x02\x03\x04\x05\x06\x07\x08\t\n\x0b\xff\xc4\x00\xb5\x10\x00\x02\x01\x03\x03\x02\x04\x03\x05\x05\x04\x04\x00\x00\x01}\x01\x02\x03\x00\x04\x11\x05\x12!1A\x06\x13Qa\x07"q\x142\x81\x91\xa1\x08#B\xb1\xc1\x15R\xd1\xf0$3br\x82\t\n\x16\x17\x18\x19\x1a%&\'()*456789:CDEFGHIJSTUVWXYZcdefghijstuvwxyz\x83\x84\x85\x86\x87\x88\x89\x8a\x92\x93\x94\x95\x96\x97\x98\x99\x9a\xa2\xa3\xa4\xa5\xa6\xa7\xa8\xa9\xaa\xb2\xb3\xb4\xb5\xb6\xb7\xb8\xb9\xba\xc2